In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline



In [110]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [111]:
#train.head()
#Vanvittig mange variable, allerede før der laves dummies.
#PCA er formentlig en god idé her, efter alt feature engineering er færdig
#train.RoofStyle.value_counts()
#test.RoofStyle.value_counts()
#train.info()

In [112]:
#test["SalePrice"]
#test(['SalePrice'], axis=1, inplace=True)
#obj_df = df.select_dtypes(include=['object']).copy()
#object_clmns = train.dtypes[train.dtypes != "int64"].index
#print(object_clmns)
#print(train.columns)
print(pd.unique(train['MSZoning']))


['RL' 'RM' 'C (all)' 'FV' 'RH']


In [113]:
#for col in train.columns:
#    if train[col].dtype != "int64":
#        bla = pd.unique(train[col]) 
#        for blatest in bla:     
#            if len(train.loc[train[col] == blatest,col]) < 50:
#                train.loc[train[col] == blatest, col] = "Test"

In [114]:
#train1 = [train['MSZoning'] == 'RH']
#mask = df['position'] >= 20
#print(train1)


In [115]:
#sns.distplot(train['SalePrice'])
#Herre skæv fordeling

In [116]:
#Log transofmration af SalePrice log(1+x)
train["SalePrice"] = np.log1p(train["SalePrice"])
#sns.distplot(train['SalePrice'])
#Meget bedre, så kan linære modeller bruges bedre. 

In [117]:
#Samler train og test set, så vi ikke skal lave rettelser 2 gange. Gemmer 
trainrækker = train.shape[0]
testrækker = test.shape[0]

#Save the 'Id' column
#train_ID = train['Id']
#test_ID = test['Id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
#train.drop("Id", axis = 1, inplace = True)
#test.drop("Id", axis = 1, inplace = True)

#Fjerner SalePrice så det ikke skal specificeres at den skal ekskluderes i alle operationer/transofmrationer af datasættet
y_train = train.SalePrice.values
#Resetter index så man kan kalde dem
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (2919, 80)


In [118]:
antal_NA= all_data.isnull().sum()
#Der er temmelig mange NA's
antal_NA.sort_values(ascending=False)[:20]

PoolQC          2909
MiscFeature     2814
Alley           2721
Fence           2348
FireplaceQu     1420
LotFrontage      486
GarageQual       159
GarageFinish     159
GarageYrBlt      159
GarageCond       159
GarageType       157
BsmtCond          82
BsmtExposure      82
BsmtQual          81
BsmtFinType2      80
BsmtFinType1      79
MasVnrType        24
MasVnrArea        23
MSZoning           4
BsmtFullBath       2
dtype: int64

In [119]:
#PoolQC er poolquality
#Skyldes det måske at husene bare ikke har en pool og der derfor bliver rapporteret NA?
len(all_data)
#Det er 2909/2919 variable der er NAN
all_data['PoolQC'].describe() #Den er ordinal/kategorisk
pd.unique(all_data['PoolQC']) 
#Går ud fra at Ex er ekstremt godt, Fa er Fantastic og Gd bare er godt. Mærkeligt at der ikke er nogle i dårlig stand?

array([nan, 'Ex', 'Fa', 'Gd'], dtype=object)

In [120]:
all_data["PoolQC"].head()
all_data["PoolQC"] = all_data["PoolQC"].fillna("Unknown")
pd.unique(all_data['PoolQC']) 

array(['Unknown', 'Ex', 'Fa', 'Gd'], dtype=object)

In [121]:
pd.unique(all_data['MiscFeature']) #Nominel feature
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("Unknown")
pd.unique(all_data['MiscFeature']) 

array(['Unknown', 'Shed', 'Gar2', 'Othr', 'TenC'], dtype=object)

In [122]:
pd.unique(all_data['Alley']) 
all_data["Alley"] = all_data["Alley"].fillna("Unknown")
pd.unique(all_data['Alley']) 

array(['Unknown', 'Grvl', 'Pave'], dtype=object)

In [123]:
pd.unique(all_data['Fence']) 
all_data["Fence"] = all_data["Fence"].fillna("Unknown")
pd.unique(all_data['Fence']) 

array(['Unknown', 'MnPrv', 'GdWo', 'GdPrv', 'MnWw'], dtype=object)

In [124]:
pd.unique(all_data['FireplaceQu'])
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("Unknown")
pd.unique(all_data['FireplaceQu']) 

array(['Unknown', 'TA', 'Gd', 'Fa', 'Ex', 'Po'], dtype=object)

In [125]:
#pd.unique(all_data['LotFrontage'])#integers, enten med median eller mean?
#Grupperer dem efter neighborhood
list(all_data.groupby("Neighborhood")["LotFrontage"])
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

In [126]:
#Samler kategoriske/nominale features og erstatter i et loop istedet
#GarageQual GarageCond GarageFinish GarageType BsmtQual BsmtExposure BsmtCond BsmtFinType2 BsmtFinType1 MasVnrType MSZoning
pd.unique(all_data['MSSubClass']) 

array([ 60,  20,  70,  50, 190,  45,  90, 120,  30,  85,  80, 160,  75,
       180,  40, 150], dtype=int64)

In [127]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual', 'SaleType',
            'BsmtExposure', 'BsmtCond','BsmtFinType2', 'BsmtFinType1', 'MasVnrType','MSZoning'):
    all_data[col] = all_data[col].fillna('Unknown')
pd.unique(all_data['Exterior2nd'])

array(['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng',
       'CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc',
       'AsphShn', 'Stone', 'Other', 'CBlock', nan], dtype=object)

In [128]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars', 'MasVnrArea', 'BsmtHalfBath', 'BsmtFullBath', 'BsmtFinSF1',
           'BsmtFinSF1', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF2'):
    all_data[col] = all_data[col].fillna(0)

In [129]:
all_data = all_data.drop(['Utilities'], axis=1)

In [130]:
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

In [131]:
antal_NA2= all_data.isnull().sum()
#Ingen NA's tilbage, yay!!!
antal_NA2.sort_values(ascending=False)[:5]

YrSold         0
Foundation     0
ExterCond      0
ExterQual      0
Exterior1st    0
dtype: int64

In [132]:
#for col in all_data.columns:
#    if all_data[col].dtype != "int64":
#        bla = pd.unique(all_data[col]) 
#        for blatest in bla:     
#            if len(all_data.loc[all_data[col] == blatest, col]) < 50:
#                all_data.loc[all_data[col] == blatest, col] = "Test"

In [133]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
#Looper over kolonner og konverterer kategoriske variable
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

Shape all_data: (2919, 79)


In [134]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)
pd.unique(all_data['FireplaceQu'])

(2919, 224)


array([5, 4, 2, 1, 0, 3], dtype=int64)

In [135]:
train = all_data[:trainrækker]
test = all_data[trainrækker:]

In [136]:
test.head()
#test.shape
#test.columns
#Hvorfor helvede har test og train lige mange kolonner??
#For fanden, husk at fjerne SalePrice ordentligt!!!!

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_Unknown,SaleType_WD
1460,896,0,0,2,2,3,3,468.0,144.0,4,...,0,0,0,0,0,0,0,0,0,1
1461,1329,0,0,2,3,3,3,923.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
1462,928,701,0,2,3,3,3,791.0,0.0,2,...,0,0,0,0,0,0,0,0,0,1
1463,926,678,0,2,3,3,3,602.0,0.0,2,...,0,0,0,0,0,0,0,0,0,1
1464,1280,0,0,2,2,3,3,263.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1


In [138]:
train["SalePrice"] = y_train
#train_ID = train['Id']
train["Id"] = train_ID
#test_ID = test['Id']
test["Id"] = test_ID

In [139]:
train.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_Unknown,SaleType_WD,SalePrice
0,856,854,0,2,3,3,3,706.0,0.0,2,...,0,0,0,0,0,0,0,0,1,12.247699
1,1262,0,0,2,3,3,1,978.0,0.0,0,...,0,0,0,0,0,0,0,0,1,12.109016
2,920,866,0,2,3,3,2,486.0,0.0,2,...,0,0,0,0,0,0,0,0,1,12.317171
3,961,756,0,2,3,1,3,216.0,0.0,0,...,0,0,0,0,0,0,0,0,1,11.849405
4,1145,1053,0,2,4,3,0,655.0,0.0,2,...,0,0,0,0,0,0,0,0,1,12.429220


In [140]:
# featuretools for automated feature engineering
import featuretools as ft

In [141]:
#Tilføjer en kolonne ti lat separare træningsset og testset.
#Laver target variable og indsætter NAN i testsæt
train['set'] = 'train'
test['set'] = 'test'
test["SalePrice"] = np.nan

In [145]:
app = train.append(test, ignore_index = False)

(1460, 226)

In [161]:
es = ft.EntitySet(id = 'Id')

In [165]:
es = es.entity_from_dataframe(entity_id = 'Id', dataframe = app, index = 'id')

2018-08-14 20:28:59,454 featuretools.entityset - WARNING    index id not found in dataframe, creating new integer column


In [160]:
es

Entityset: test
  Entities:
    test [Rows: 2919, Columns: 227]
  Relationships:
    No relationships

In [167]:
default_agg_primitives =  ["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
default_trans_primitives =  ["day", "year", "month", "weekday", "haversine", "numwords", "characters"]

# DFS with specified primitives
feature_matrix, feature_names = ft.dfs(entityset = es, target_entity = 'Id',
                                       trans_primitives = default_trans_primitives,
                                       agg_primitives=default_agg_primitives, 
                                        max_depth = 3, features_only=False, verbose = True)

print('%d Total Features' % len(feature_names))

KeyboardInterrupt: 

In [168]:
feature_matrix_spec, feature_names_spec = ft.dfs(entityset = es, target_entity = 'Id',  
                                                 agg_primitives = ['sum', 'count', 'min', 'max', 'mean', 'mode'], 
                                                 max_depth = 2, features_only = False, verbose = True)

Built 225 features
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████████████████████████████████████| Calculated: 11/11 chunks


In [171]:
es = ft.EntitySet('Transactions')

es.entity_from_dataframe(dataframe=app,
                         entity_id='hvad',
                         index='Id')

#fm, features = ft.dfs(entityset=es, 
#                      target_entity='log',
#                      trans_primitives=['day', 'weekday', 'month'])

AssertionError: Index is not unique on dataframe (Entity hvad)